# **Tworzenie API we Flasku – Wprowadzenie**

W tym ćwiczeniu nauczysz się, jak stworzyć proste API w Flasku, uruchomić je, wysyłać do niego zapytania oraz wykorzystać model decyzyjny w oparciu o podstawową regułę logiczną.

## **1️⃣ Tworzenie podstawowego API**
Najpierw utworzymy podstawową aplikację Flask.

### **Zapisanie kodu API do pliku**
W Jupyter Notebooku użyj magicznej komendy `%%file`, aby zapisać kod podstawowej aplikacji flask do pliku `app.py`: Kod znajdziesz na [cw1](https://sebkaz-teaching.github.io/RTA_2025/cw1.html)
Jako tekst do wyświetlenie strony głównej użyj `Witaj w moim API!`.

In [1]:
%%file app.py

from flask import Flask

# Create a flask
app = Flask(__name__)

# Create an API end point
@app.route('/')
def home():
    return "Witaj w moim API!"

if __name__ == '__main__':
    app.run(debug=True)

Writing app.py


Teraz uruchom API w terminalu, wpisując:
```sh
python app.py
```
Flask uruchomi serwer lokalnie pod adresem `http://127.0.0.1:5000/`.

In [2]:
!python app.py

^C


### **Sprawdzenie działania API**
W Jupyter Notebooku wykonaj zapytanie GET do strony głównej. Na podstawie pola `status_code` napisz wyrażenie warunkowe które dla status_code 200 wyświetli zawartość odpowiedzi (z pola `content`).

In [3]:
import requests
response = requests.get("http://127.0.0.1:5000/") 

if response.status_code ==200:
    print(response.content.decode())

Witaj w moim API!


Jeśli wszystko działa poprawnie, zobaczysz komunikat `Witaj w moim API!`.
---

## **2️⃣ Dodanie nowej podstrony**
Dodajmy nową podstronę `mojastrona`, która zwróci komunikat `To jest moja strona!`.

In [4]:
%%file app.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

if __name__ == '__main__':
    app.run(debug=True)

Overwriting app.py


Ponownie uruchom API i wykonaj zapytanie do strony `"http://127.0.0.1:5000/mojastrona"`:

In [5]:
response = requests.get("http://127.0.0.1:5000/mojastrona")
if response.status_code ==200:
    print(response.content.decode())

In [6]:
!python app.py

^C


Powinieneś zobaczyć: `To jest moja strona!`

---

## **3️⃣ Automatyczne uruchamianie serwera z Jupyter Notebook**
Zamknij wcześniej uruchomiony serwer (`Ctrl+C` w terminalu) i uruchom go ponownie bezpośrednio z Jupyter Notebook, korzystając z `subprocess.Popen`:

In [7]:
import subprocess

server = subprocess.Popen(["python", "app.py"])

Po testach zamknij serwer wykorzystując metodę `kill`:


In [8]:
server.kill()

---

## **4️⃣ Obsługa parametrów w adresie URL**
Dodajemy nową podstronę `/hello`, która będzie przyjmować parametr `name`.

Edytuj `app.py`, dodając odpowiedni  kod


In [9]:
%%file app.py

from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

@app.route('/hello')
def hello():
    name = request.args.get('name')
    if name:
        return f"Hello {name}!"
    else:
        return "Hello!"
    
if __name__ == '__main__':
    app.run(debug=True)

Overwriting app.py


Uruchom serwer i sprawdź działanie API:
```python
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"
```

---

In [10]:
import requests

res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content.decode())  # Powinno zwrócić "Hello!"

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



In [11]:
res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content.decode())  # Powinno zwrócić "Hello Sebastian!"

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



## **5️⃣ Tworzenie API z prostym modelem ML**
Stworzymy nową podstronę `/api/v1.0/predict`, która przyjmuje dwie liczby i zwraca wynik reguły decyzyjnej:
- Jeśli suma dwóch liczb jest większa niż 5.8, zwraca `1`.
- W przeciwnym razie zwraca `0`.


In [12]:
%%file app.py

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

@app.route('/hello')
def hello():
    name = request.args.get('name')
    if name:
        return f"Hello {name}!"
    else:
        return "Hello!"

@app.route('/api/v1.0/predict')
def predict():
    try:
        x1 = float(request.args.get('num1'))
        x2 = float(request.args.get('num2'))
    except (TypeError, ValueError):
        return jsonify({"error": "Błędne dane wejściowe. Podaj liczby num1 i num2"}), 400

    wynik = 1 if (x1 + x2) > 5.8 else 0

    return jsonify({
        "prediction": wynik,
        "features": {
            "num1": x1,
            "num2": x2
        }
    })

if __name__ == '__main__':
    app.run(debug=True)

Overwriting app.py


Sprawdź działanie API:
```python
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}
```

---

## **Podsumowanie**
Po wykonaniu tego ćwiczenia studenci będą umieli:
✅ Tworzyć podstawowe API w Flasku.  
✅ Dodawać podstrony i obsługiwać parametry URL.  
✅ Wysyłać zapytania GET i analizować odpowiedzi.  
✅ Automatycznie uruchamiać serwer z Jupyter Notebook.  
✅ Implementować prosty model decyzyjny w API.  

Gotowi na kolejne wyzwania? 🚀

In [13]:
import subprocess
import time

server = subprocess.Popen(["python", "app.py"])
time.sleep(2)

In [14]:
import requests

res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())

{'features': {'num1': 3.0, 'num2': 4.0}, 'prediction': 1}


In [15]:
%%writefile requirements.txt
flask
requests

Writing requirements.txt


In [16]:
%%writefile README.md
# REST API zawierający regułę decyzyjną

Projekt - aplikacja Flask z czterema endpointami

# Endpointy
- `/` – strona główna
- `/mojastrona` – zwykła podstrona
- `/hello?name=TwojeImie` – przyjmuje parametr `name`
- `/api/v1.0/predict?num1=3&num2=4` – jeśli suma > 5.8, zwraca 1, inaczej 0

## Jak uruchomić:
1. Zainstaluj zależności:
    
pip install -r requirements.txt

2. Uruchom aplikację:
    
python app.py

3. Otwórz przeglądarkę i przetestuj podstrony:

- Strona główna: http://127.0.0.1:5000/
- Moja strona: http://127.0.0.1:5000/mojastrona
- Hello: http://127.0.0.1:5000/hello?name=TwojeImie
- Predykcja: http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4

Writing README.md


In [17]:
%%writefile Dockerfile
FROM python:3.11-slim-buster

WORKDIR /app

COPY requirements.txt .

RUN pip install -r requirements.txt

COPY app.py .

ENV FLASK_APP=app.py

EXPOSE 5000

CMD ["flask", "run", "--host=0.0.0.0", "--port=5000"]


Writing Dockerfile
